In [ ]:
pip install google-auth google-auth-httplib2 google-api-python-client pandas


In [ ]:
import csv
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Path to your JSON file with Google Calendar API credentials
json_file = "your-json-file.json"

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_service():
    credentials = service_account.Credentials.from_service_account_file(
        json_file, scopes=SCOPES)
    return build('calendar', 'v3', credentials=credentials)

def create_event(service, summary, start_date, description):
    # Extract month and day from the date
    start_date_parts = start_date.split('-')
    year, month, day = start_date_parts[0], start_date_parts[1], start_date_parts[2]
    
    # Create event with recurring rule
    event = {
        'summary': summary,
        'description': description,
        'start': {
            'date': start_date,
            'timeZone': 'America/New_York',  # Adjust as needed
        },
        'end': {
            'date': start_date,
            'timeZone': 'America/New_York',  # Adjust as needed
        },
        'recurrence': [
            f'RRULE:FREQ=YEARLY;BYMONTH={month};BYMONTHDAY={day}'
        ]
    }
    
    try:
        event_response = service.events().insert(calendarId='youremailid@gmail.com', body=event).execute()  #add your email address
        print(f'Event created: {event_response.get("htmlLink")}')
    except Exception as e:
        print(f'An error occurred: {e}')

def import_events_from_csv(csv_file):
    service = get_service()
    
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            summary = row['Subject']
            start_date = row['Start Date']  # e.g., '2024-08-10'
            description = row['Description']
            print(f'Processing event: {summary} on {start_date}')
            create_event(service, summary, start_date, description)

# Main execution
if __name__ == '__main__':
    import_events_from_csv("your_csv_file.csv") #add your csv file
